# StableDiffusion模型LoRA微调训练

PAI提供了丰富的，面向图像、语音、文本等多个领域，基于预训练模型的算法，能够支持用户简单便捷地完成预训练模型的微调训练。

本文档将介绍如何通过PAI Python SDK调用PAI平台上提供的 `kohya_lora_trainer` 训练算法，他基于开源社区提供的`kohya_ss`构建，支持StableDiffusion模型[LoRA(Low-Rank Adaptation)](https://arxiv.org/abs/2106.09685)高效微调训练，从而获得可以生成特定领域风格图片的LoRA模型。


## 准备工作

通过以下命令安装PAI Python SDK。

In [ ]:
!python -m pip install --upgrade alipai


SDK 需要配置访问阿里云服务需要的 AccessKey，以及当前使用的工作空间和OSS Bucket。在 PAI Python SDK 安装之后，通过在 **命令行终端** 中执行以下命令，按照引导配置密钥，工作空间等信息。


```shell

# 以下命令，请在 命令行终端 中执行.

python -m pai.toolkit.config

```

In [ ]:
from pai.session import get_default_session, setup_default_session

# 用户通过以下的方式配置SDK的session.
sess = get_default_session()
if not sess:
    print("config session")
    sess = setup_default_session(
        region_id="<REGION_ID>",  # 例如：cn-beijing
        workspace_id="<WORKSPACE_ID>",  # 例如：12345
        oss_bucket_name="<OSS_BUCKET_NAME>",
    )
# 将当前的配置持久化到 ~/.pai/config.json，SDK默认从对应的路径读取配置初始化默认session。
sess.save_config()

print(sess.workspace_name)

我们可以通过以下代码验证当前的配置。

In [ ]:
import pai
from pai.session import get_default_session

print(pai.__version__)
sess = get_default_session()
print(sess.workspace_id)
print(sess.workspace_name)

## 算法简介


通过 `pai.estimator.AlgorithmEstimator` 对象，用户可以轻松的在 SDK 中调用 PAI 平台的已注册算法（指定 `algorithm_provider` 为 `pai` 便可以调用 PAI 平台所提供的官方算法）。


获取`kohya_lora_trainer`算法示例代码如下：



In [4]:
from pai.estimator import AlgorithmEstimator

# 创建 AlgorithmEstimator 对象
est = AlgorithmEstimator(
    algorithm_name="kohya_lora_trainer",  # 算法名称
    algorithm_version="v0.1.0",  # 算法版本
    algorithm_provider="pai",  # 算法提供者，制定为 pai 以调用 PAI 平台所提供的官方算法
    base_job_name="SD_kohya_lora_training",  # 任务名称
    # 用户可以查看 est.supported_instance_types 来获取支持的实例类型；
    # 用户可以根据自己的需求选择合适的实例类型，不填写则会选择默认的实例类型
    # instance_type="ecs.gn6v-c8g1.2xlarge",
)



通过`AlgorithmEstimator.hyperparameter_definitions`，`AlgorithmEstimator.input_channel_definitions`等属性，我们可以获取算法支持的参数定义，输入定义等信息。

查看`kohya_lora_trainer`训练算法支持的超参：

In [5]:
from IPython.display import HTML, display


def table_display(data):
    display(
        HTML(
            "<table><tr>{}</tr></table>".format(
                "</tr><tr>".join(
                    '<td style="text-align:left;">{}</td>'.format(
                        '</td><td style="text-align:left;">'.join(str(_) for _ in row)
                    )
                    for row in data
                )
            )
        )
    )


hp_defs = [["HyperParameter Name", "Type", "DefaultValue", "Required", "Description"]]

for ch in est.hyperparameter_definitions:
    hp_defs.append(
        [
            ch["Name"],
            ch["Type"],
            est.hyperparameters.get(ch["Name"], ch["DefaultValue"]),
            ch["Required"],
            ch["Description"],
        ]
    )

table_display(hp_defs)

HyperParameter Name,Type,DefaultValue,Required,Description
max_epochs,Int,10,True,Total train epochs.
batch_size,Int,1,True,Train batch size per gpu.
learning_rate,Float,0.0001,True,Initial learning rate.
resolution,String,512,False,"image resolution in training (""size"" or ""width,height"")"
network_module,String,networks.lora,True,"Network module name. Accept Values: [""networks.lora"", ""networks.dylora""]"
network_dim,Int,32,False,"The number of dimensions of LoRA. The greater the number, the greater the expressive power, but the memory and time required for learning also increase. In addition, it seems that it is not good to increase it blindly.."
network_alpha,Float,32.0,False,Alpha for LoRA weight scaling(if 1 same as network_dim for same behavior as old version)
network_args,String,,False,"Optional, default None. Additional argmuments for network, such as '""conv_dim=16"" ""unit=4""'"
save_every_n_epochs,Int,5,False,Save checkpoint interval by epoch.
max_token_length,Int,225,False,"Max token length of text encoder (default for 75, 150 or 225)."


通过`est.hyperparameters`属性和`est.set_hyperparameters` 方法，用户可以查看当前算法配置使用的超参，以及修改超参配置。

In [ ]:
print(est.hyperparameters)

# 配置算法超参
est.set_hyperparameters(
    learning_rate=2e-04,
    batch_size=4,
)

print(est.hyperparameters)

Tips: 使用DyLoRA进行训练


当前算法也支持使用[DyLoRA: Parameter Efficient Tuning of Pre-trained Models using Dynamic Search-Free Low-Rank Adaptation](https://arxiv.org/abs/2210.07558)。该论文提出LoRA的rank并不是越高越好，而是需要根据模型、数据集、任务等因素来寻找合适的rank。使用DyLoRA，可以同时在指定的维度(rank)下学习多种rank的LoRA，从而省去了寻找最佳rank的麻烦。

```python

est.set_hyperparameters(**{
	"network_module": "networks.dylora",
	"network_args": '"unit=4"',
	"network_dim": 16,
})
```

例如，当使用以上的超参配置时，dim=16、unit=4进行学习时，可以学习和提取4个rank的LoRA，即4、8、12和16。通过在每个提取的模型中生成图像并进行比较，可以选择最佳rank的LoRA。 如果未指定unit，则默认为unit=1。



## 准备训练数据

训练数据需要准备在OSS上，具体的数据格式可参考：[image_folder_structure.md](https://github.com/bmaltais/kohya_ss/blob/master/docs/image_folder_structure.md) 。

在训练目录文件夹内需要准备供模型训练的图片和txt标注文件，txt标注文件来注明每张图片对应的prompt文本。当txt标注文件不存在时，模型将使用图片目录名作为作为prompt文本。

> 图像标注文件的生成可使用PAI快速开始提供“image-captioning”系列模型。

数据示例：

```plain
├──train_data
| ├── 30_dog
| | |
| | ├── image1.jpg
| | ├── image1.txt
| | ├── image2.png
| | └── image2.txt
| | └── ...
```

`imgxx.txt`的文件内容如下：

```text
a dog is running
```


PAI提供了一份简单的训练数据示例，可以直接用于kohya_lora_trainer的训练。

In [ ]:
train_data = (
    "oss://pai-quickstart-{region}/aigclib/datasets/kohya_ss/lora/v0.1.0/train/".format(
        region=sess.region_id
    ),
)

用户可以通过以下的代码，下载PAI提供训练数据到本地查看。

```python

from pai.common.oss_util import download
download(train_data, "./train_data")

```




## 提交训练作业


`kohya_lora_trainer` 算法要求提供的输入：
- `pretrained_model` 输入：
  
预训练模型，相应的目录下应该包含一个StableDiffusion格式的预训练模型，可以是`.safetensors`文件格式或是一个`.ckpt`文件格式，或是HuggingFace Diffusers支持的预训练模型格式。

- `train` 输入:

训练的输入数据，默认是OSS URI格式(`oss://<Bucket>/path/to/train/data/`)


可以通过以下代码查看训练算法的输入输出：


In [ ]:
io_defs = [["Channel Name", "Input/Output", "Required", "Description"]]
for ch in est.input_channel_definitions:
    io_defs.append([ch["Name"], "Input", ch["Required"], ch["Description"]])
for ch in est.output_channel_definitions:
    io_defs.append([ch["Name"], "Output", ch["Required"], ch["Description"]])


table_display(io_defs)

在本示例中，我们将使用PAI提供的一份简单训练数据集和预训练模型进行训练。 通过 `AlgorithmEstimator.fit()` 方法我们可以提交训练任务。SDK会打印训练作业的控制台URL，用户可以通过该URL查看训练作业的详细信息以及日志。


In [ ]:
region = sess.region_id
# PAI提供的训练数据集和预训练模型(SD1.5)
train = (f"oss://pai-quickstart-{region}/aigclib/datasets/kohya_ss/lora/v0.1.0/train/",)
pretrained_model = f"oss://pai-quickstart-{region}/aigclib/models/custom_civitai_models/sd1.5/v1-5-pruned.safetensors"

# 提交训练作业
est.fit(inputs={"pretrained_model": pretrained_model, "train": train})


任务结束后，用户可以通过 `AlgorithmEstimator.model_data()` 方法来查看产出模型的地址信息。

In [ ]:
# 查看输出的模型地址
print(est.model_data())

模型默认存储在OSS上，用户可以通过以下代码将模型下载到本地。

```python

from pai.common.oss_util import download

download(est.model_data(), "./model/)


```